![foto](https://esports.eldesmarque.com/wp-content/uploads/2019/09/LoL2-681x382.jpg)

En este proyecto se va a realizar un análisis de partidas de alto Elo (considerando este Diamante o superior) de League of Legends. League of Legends (LOL) es un juego multijugador con una comunidad formada por millones de jugadores en todo el mundo, la cual es capaz de condicionar la evolución del juego, sin pasar por alto su versión profesional ,considerada como uno de los juegos más importantes dentro de la escena de e-sports. Con todo esto, estamso analizando un juego de gran repercusión, el cual está sometido a constantes cambios por parte de sus desarrolladores con parches y actualizaciones del juego que bien pueden ser simples ajustes o cambios más severos que afecten parte de la jugabilidad en ciertos aspectos del juego.

El análisis se va a centrar en el early game (o juego temprano) puesto que los datos a tratar son de los primeros 10 minutos de cada partida. Estos datos están recopilados en un archivo .csv los cuales serán procesados scripts de Python y distintas herramientas enfocadas en el analisis de datos (librerias, herramientas estadisticas...)

## **1. Importación previa de las librerías necesarias**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.graph_objects as go

## **2. Cargamos el dataset**

In [ ]:
df = pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")

### 2.1 Muestra de los primeros valores del archivo

In [ ]:
df.head()

### 2.1.1 Variable de salida

La variable que utilizaremos para clasificar los datos será **blueWins**. Donde:
- **1** nos indica que el equipo azul **gana la partida**.
- **0** nos indica que  **es el equipo rojo**  el que gana la partida.

Como se aprecia, seria redundante una columna con **redWins** ya que serian datos complementarios a **blueWins**

### 2.1.2 Variables de entrada

Como has podido observar en el apartado anterior, para predecir quién va a ganar la partida, tenemos un total de **40 variables** asociadas a la mecánica del juego. En total, contamos con una muestra de **9879 partidas.** 

### 2.1.3 Columnas
No se distingue entre lado azul y lado rojo
* blueWins : Victorias y derrotas del equipo azul
* WardsPlaced : Número de wards colocados
* WardsDestroyed : Número de wards destruidos
* FirstBlood :Primera muerte de la partida, siendo 1 para el equipo azul y 0 para el equipo rojo
* Kills : Número de asesinatos 
* Deaths : Número de muertes
* Assists : Número de asistencias 
* EliteMonsters : Número de monstruos épicos asesinados (Dragones y Heraldos)
* Dragons : Número de dragones asesinados por el equipo
* Heralds : Número de Heraldos asesinados por el equipo
* TowersDestroyed : Número de torres destruidas 
* TotalGold : Oro global del equipo
* AvgLevel : Media de nivel del equipo
* TotalExperience : Experiencia total del equipo
* TotalMinionsKilled : Minions asesinados totales (CS)
* TotalJungleMinionsKilled : Monstruos totales de la jungla asesinados por el equipo
* GoldDiff : Comparación del oro global por equipos
* ExperienceDiff : Diferencias de experiencia total por equipos
* CSPerMin : Minions por minuto del equipo
* GoldPerMin : Oro por minuto del equipo

## **3. Análisis del dataset:**

![Immune](https://i.imgur.com/FBQ64Uz.png)

Para entender la distribución de los datos, vamos a observar:

- La cantidad de datos (count)
- La distribución de los datos mediante cuartiles (25%,50%,75%)
- Media de los datos (mean)
- Mínimos y máximos (min, max)
- Varianza (std)
- ...

Para extraer todos estos datos se usará la función **.describe()** para analizar la distribucion de los datos.

In [ ]:
df.describe()

## **4. Representación de datos:**

 **4.1 Importancia del lado del mapa:**

In [ ]:
labels = ['Red Team Wins', 'Blue Team Wins']
red_wins = len(df.blueWins[df['blueWins']==0])
blue_wins = len(df.blueWins[df['blueWins']==1])
wins = (red_wins,blue_wins)
fig = plt.figure()
barPlot = plt.bar(labels,wins)
plt.ylabel('Number of Wins')
plt.title('Outcome Distribution')
barPlot[0].set_color('red')
barPlot[1].set_color('blue')
plt.show()

El mapa de League of Legends esta representado en espejo, ubicandose la línea divisoria en la diagonal. Por tanto, no existe ventaja de campo demostrada con un 50% de victoria cada una, siendo indiferente en que lado del mapa se empiece para conseguir la victoria. Sin embargo, aunque no hay ventajas estadisticas, ciertos equipos ajustan su composicion o estilo de juego para adaptarse al lado del campo que les ha tocado (coberturas de objetivos, tendencias de gankeo...)

 **4.2 Visión del mapa**

La visión del mapa es un factor que puede controlarse por parte de los jugadores en cada partida. Por defecto solo se tiene la vision proporcionada por las estructuras (torres), la de cada jugador del equipo en cada momento y la proporcionada por los subditos, quedando el resto del mapa oculto. Para la gestión de la vision, en el juego existen dos elementos: **El Ward y la lente del Oráculo**. Siendo el primero para colocar vision y el segundo para retirar visión enemiga. La visión puede revelar información importante sobre el enemigo que se puede utilizar para el devenir de los movimientos de la partida. 

![fotomapa](https://pm1.narvii.com/6760/eebaf2b07dfeafd4a184bc89038e969637166f8dv2_hq.jpg)

In [ ]:
print('Media de Wards colocados cuando el equipo AZUL gana : %.5f' %df[df.blueWins == 1]['blueWardsPlaced'].mean())

In [ ]:
print('Media de Wards colocados cuando el equipo AZUL pierde (Es decir, ROJO gana) : %.5f' %df[df.blueWins == 0]['blueWardsPlaced'].mean())

In [ ]:
color_discrete_map = {'Blue': 'rgb(122, 148, 231)', 'Red': 'rgb(255, 105, 97)'}


X1 = df[[ 'blueWardsDestroyed']].copy()
X1.columns = ['WardsDestroyed']
X1 = X1.astype(float)
X1['Side'] = 'Blue'
X2 = df[['redWardsDestroyed']].copy()
X2.columns = ['WardsDestroyed']
X2 = X2.astype(float)
X2['Side'] = 'Red'
info = pd.concat([X1, X2], ignore_index = True).sample(2000)
info




fig1 = px.violin(info, y="WardsDestroyed", color = 'Side',  box=True, points='all', color_discrete_map=color_discrete_map, title = 'Wards Destroyed por equipos')

layout = go.Layout(
    yaxis=dict(
        range=[5, 45]
    ),
    xaxis=dict(
        range=[100, 200]
    )
)

fig2 = px.violin(info, y="WardsDestroyed", color = 'Side',  box=True, points='all', color_discrete_map=color_discrete_map, title = 'Wards Destroyed por equipos (Ampliado)')
fig2.update_layout(
    yaxis=dict(
        range=[0, 10]
    )
)
fig1.show()
fig2.show()

Una vez analizados los elementos de vision (wards) colocados y los wards destruidos, no se aprecia una ventaja clara relacionada al trabajo de la vision del equipo. No podemos concluir que la vision no tiene especial relevancia para ganar la partida ya que se tratan de jugadores experimentados y probablemente los 10 jugadores tienen interiorizado y optimizado el uso de la vision. Por tanto una mayor cantidad de wards no implica la victoria pero el descuido de este aspecto del juego podria condicionarla.

**4.3 Oro del equipo**

El oro es esencial en el juego, ya que permite a los jugadores comprar nuevos objetos o mejorar los que ya tienen para sacarle el mayor rendimiento a sus personajes. Hay cuatro formas de ganar oro en la partida: oro pasivo, matando súbditos o monstruos de jungla, matando a otros campeones y por destruccion de objetivos (torres, inhibidores, dragones...)

In [ ]:
x = df['blueWins']
y = df['blueTotalGold']
blue_loses = len(df.blueWins[df['blueWins']==0])
blue_wins = len(df.blueWins[df['blueWins']==1])
blue = (blue_loses,blue_wins)
fig = plt.figure()
labels = ['Blue Team Loses', 'Blue Team Wins']
barPlot = plt.bar(labels,blue)
plt.ylabel('Oro total del equipo')
plt.bar(x, y)
plt.xticks(range(0,2))
plt.show()

Los resultados nos indican que una mayor cantidad de oro global, favorece a la victoria. Es algo logico ya que se puede llegar al "power pike" del campeón antes con la compra de objetos en la fase temprana.

**4.3.1 Primera Sangre**

La primera sangre se produce cuando ocurre el primer asesinato de la partida. Este no puede ser considerado como "una kill" mas ya que proporciona un 33% mas de oro al asesino y una mayor recompensa a los asistentes que han ayudado a la consecución de esta.

In [ ]:
prob_wins = df.groupby('blueFirstBlood')['blueWins'].mean()
prob_wins = prob_wins.reset_index()
prob_wins.columns = ['blueFirstBlood', 'Probablidad de victoria']

plt.figure(figsize=(8,5))
ax = sns.barplot(x="blueFirstBlood", y="Probablidad de victoria", data=prob_wins)

Los datos reflejan que la primera sangre tiene un impacto significativo en la probabilidad de victoria del equipo que la consigue. Probablemente por la compra de objetos antes que el otro equipo y la experiencia perdida del jugador que es asesinado en los primeros compases de la partida.

**4.4 Importancia de los Objetivos**

Aunque no lo parezca por su carácter competitivo, League of Legends no es un juego que se base exclusivamente a asesinar a los contrarios. Para ganar la partida se deben de destruir objetivos hasta llegar al nexo enemigo. Estos objetivos son Torres, Inhibidores, Dragones, Heraldos y Monstruos épicos (Nashor y Elder). Estos últimos como salen despues del min 20 no serán analizados.

In [ ]:
col = ['blueTowersDestroyed','blueDragons','blueHeralds','redTowersDestroyed','redDragons','redHeralds']
tmp1 = df[col[0:3]].copy()
tmp1.columns = ['TowersDestroyed','Dragons','Heralds']
tmp1['Team'] = 'Blue'
tmp2 = df[col[3:6]].copy()
tmp2.columns = ['TowersDestroyed','Dragons','Heralds']
tmp2['Team'] = 'Red'

data = pd.concat([tmp1, tmp2], ignore_index = True)
data = data.groupby('Team').mean().reset_index()
data=pd.melt(data, id_vars=['Team'], value_vars=['TowersDestroyed','Dragons','Heralds'])
data.columns = ['Team','Objetives','Mean']

fig = px.bar(data, x="Team", y="Mean", color="Team", color_discrete_map=color_discrete_map, #colores definidos en funciones anteriores
             facet_col="Objetives", title = 'Toma de objetivos en el juego temprano'
            )
fig.show()

Si analizamos los objetivos de la partida, vemos como el quipo azul suele tomar ligeramente mas torres que el equipo rojo. Esto puede estar condicionado con el resto de objetivos que coinciden simultaneamente en la partida. El equipo rojo tiende a llevarse los dragones, pudiendo ser influenciado por el lado del mapa. Esta tendencia hace que el equipo azul contrarreste esta jugada con la conquista del heraldo para igualar la contienda tanto en oro global como por objetivos.

**5. Analizar las relaciones de las variables del dataset**

In [ ]:
df.corr()

In [ ]:
# Matriz de correlación con valores absolutos

corr = df[[col for col in df.columns if 'blue' in col ]].corr().abs()

upper = corr.where(np.triu(np.ones(corr.shape), 
                                  k=1).astype(np.bool))

# Obtención de características con una correlación superior a 0,75
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

# Pintamos la nueva matriz de correlación del equipo AZUL, que es igual que para la del equipo ROJO

plt.figure(figsize=(15,10))
t = sns.heatmap(corr, annot = True, linewidths=.3, cmap ='coolwarm')
t.set_title('Correlacion del equipo Azul')
plt.show()

Cuanto más cercano es el valor a 1 o -1, más correlación (lineal) existe.

In [ ]:
corr = df[[col for col in df.columns if 'blue' in col 
           and col != "blueWardsPlaced" 
           and col != "blueWardsDestroyed" 
           and col != "blueTotalJungleMinionsKilled" 
           and col != "blueHeralds" 
           and col != "blueTowersDestroyed"]].corr().abs()


to_drop = [column for column in upper.columns if any(
        upper[column] > 0.75)]

plt.figure(figsize=(15,10))
t = sns.heatmap(corr, annot = True, linewidths=.3, cmap ='coolwarm')
t.set_title('Correlacion del equipo Azul')
plt.show()

In [ ]:
X = df.drop(to_drop, axis=1)
X = X.drop(["blueWardsPlaced", "blueWardsDestroyed",
           "blueTowersDestroyed", "blueTotalJungleMinionsKilled", "blueHeralds", "blueTowersDestroyed", "redHeralds", "redTowersDestroyed",
           "redWardsPlaced", "redWardsDestroyed",
           "redTowersDestroyed", "redTotalJungleMinionsKilled"], axis = 1)

X.to_csv('./validdataset.csv', index = False)

**5.1 Creacion del conjunto de datos de entrenamiento y del conjunto de datos de test.**

In [ ]:
# Variables dependientes e independientes
X = X.drop(['gameId'], axis = 1)
Y = df["blueWins"]

In [ ]:
from sklearn.model_selection import train_test_split
testSize = 0.2
x_train, x_test, y_train, y_test = train_test_split(
        X, Y, test_size=testSize, random_state = 4)

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train,y_train)

In [ ]:
logistic_prob = logistic.predict_proba(x_test)
logistic_prob = logistic_prob[:,-1]

**5.2  Matriz de Confusión**

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
from tabulate import tabulate


# ROC Curve

logistic_fpr, logistic_tpr, logistic_thresholds = roc_curve(y_test, logistic_prob)

# AUC Scores

logistic_auc = roc_auc_score(y_test, logistic_prob)

# Best threshold for Logistic Regression Model

equation_logistic = np.sqrt(logistic_tpr * (1-logistic_fpr))
index_logistic = np.argmax(equation_logistic)


# Logistic Regression Model predictions
logistic_prediction = []
for i in logistic_prob:
    if i>= logistic_thresholds[index_logistic]:
        i = 1
        logistic_prediction.append(i)
    else:
        i=0
        logistic_prediction.append(i)


logistic_acc    = accuracy_score(logistic_prediction, y_test)
logistic_prec   = precision_score(logistic_prediction, y_test)
logistic_recall = recall_score(logistic_prediction, y_test)


# Confusion Matrix for Logistic Regression Model
logistic_tn, logistic_fp, logistic_fn, logistic_tp = confusion_matrix(
        y_test, logistic_prediction).ravel()


table = [
['Logistic Regression   ', logistic_acc, logistic_prec, logistic_recall, logistic_tp, logistic_fp, logistic_tn, logistic_fn]
        ] 
print (tabulate(table, headers=["Model", "Accuracy", "Precision", "Recall",
                               "TP", "FP", "TN", "FN"]))

**Conclusión**

Después de esta modelización de datos con el 96% de precisión se puede sacar como conclusión:

No hay ventaja de mapa entre lado rojo y lado azul.

Tambien otro dato significativo es que las Kills no son el factor más determinante para ganar las partidas, tiene más peso el Oro y la experiencia ganadas. Por tanto hay que aprender a moverse bien por el mapa para optimizar la ganancia de ambas en vez de unicamente centrarnos en matar, ya que en esos momentos son cuando el jugador puede morir al exponerse y tirar la partida. Una manera que parece que tiene el juego de recompensar el trabajo en equipo es el reparto de oro de las asistencias, por tanto no supone el mismo resultado 3 \"solo\" kills que 3 kills con X asistencias, mejorando el oro y la experiencia globales, generando un efecto bola de nieve "snowball".

Una cantidad mayor de wards no implica un mayor porcentaje de victoria, si bien es importante trabajar la vision puesto que si no lo haces tu, el equipo contrario si lo hace y puede sacar ventaja por obtener información proporcionada por sus Wards.
        
La primera sangre no es determinante para el devenir de la partida pero proporciona una gran ventaja en el early game al equipo que la consigue puesto es importante conseguirla pero tambien no concederla.

Los objetivos tienen un gran peso en la partida, ya que nos proporciona Oro, experiencia y bonus en nuestras estadisticas de campeon. El equipo rojo parece que tiene una ventaja en la pelea de dragones. El equipo azul suele hacer intercambio de objetivos y conseguir mas Heraldos.

Realizado por aficionado a League of legends que esta iniciandose en la infórmatica y en el análisis de datos.

In [ ]:
#################################
#Función auxiliar (NO MODIFICAR):
#################################

def eval_best_model(final_model,team_name):
    data_valid = pd.read_csv('validdataset.csv')
    
    X_valid = data_valid.drop('gameId',axis = 1)   
       
    y_valid_pred = final_model.predict(X_valid)
    
    submission = pd.DataFrame({ 'id': data_valid['gameId'],
                                'output': y_valid_pred })
    submission.to_csv('./submission_team_'+team_name+'.csv', index = False)
    return(submission)

In [ ]:
eval_best_model(logistic,'AndresNaranjoJimenez')